In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from timeit import default_timer as timer

import importlib
import utils
import networks

# change the default figure size
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams.update({'font.size': 16})

# Compare AWIPINN, IPINN, AWPINN and PINN

In [ ]:
# Function to load average loss data for a given configuration
key = 'MSE_Loss'

def load_average_loss(config):
    loss_data = pd.read_csv(f'test_final_performance/{config}/average_loss.csv')
    return loss_data[f'Average_{key}'].to_numpy()

# Function to create and save a loss plot for a given configuration
def create_loss_plot(config, num_epochs=10000):
    loss_list1 = load_average_loss(f'pinn/{config}')
    loss_list2 = load_average_loss(f'ipinn/{config}')
    loss_list3 = load_average_loss(f'awpinn/{config}')
    loss_list4 = load_average_loss(f'awipinn/{config}')
    
    fig = plt.figure(figsize=(12, 9))
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list1[:num_epochs]], color='blue', label='pinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list2[:num_epochs]], color='red', label='ipinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list3[:num_epochs]], color='green', label='awpinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list4[:num_epochs]], color='cyan', label='awipinn')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'test_plot/{config}_normal.png', bbox_inches='tight')

# List of configurations to create plots for
configurations = ['1-1-1', '2-1-1']

# Create and save loss plots for each configuration
for config in configurations:
    create_loss_plot(config)

In [ ]:
# Function to load average loss data for a given configuration
key = 'MSE_Loss'

def load_average_loss(config):
    loss_data = pd.read_csv(f'test_diff_nn/25neurons/{config}/average_loss.csv')
    return loss_data[f'Average_{key}'].to_numpy()

# Function to create and save a loss plot for a given configuration
def create_loss_plot(config, num_epochs=10000):
    loss_list1 = load_average_loss(f'pinn/{config}')
    loss_list2 = load_average_loss(f'ipinn/{config}')
    loss_list3 = load_average_loss(f'awpinn/{config}')
    loss_list4 = load_average_loss(f'awipinn/{config}')
    
    fig = plt.figure(figsize=(12, 9))
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list1[:num_epochs]], color='blue', label='pinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list2[:num_epochs]], color='red', label='ipinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list3[:num_epochs]], color='green', label='awpinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list4[:num_epochs]], color='cyan', label='awipinn')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'test_plot/{config}_25neurons.png', bbox_inches='tight')

# List of configurations to create plots for
configurations = ['1-1-1', '2-1-1']

# Create and save loss plots for each configuration
for config in configurations:
    create_loss_plot(config)

In [ ]:
# Function to load average loss data for a given configuration
key = 'MSE_Loss'

def load_average_loss(config):
    loss_data = pd.read_csv(f'test_diff_nn/4_layers/{config}/average_loss.csv')
    return loss_data[f'Average_{key}'].to_numpy()

# Function to create and save a loss plot for a given configuration
def create_loss_plot(config, num_epochs=10000):
    loss_list1 = load_average_loss(f'pinn/{config}')
    loss_list2 = load_average_loss(f'ipinn/{config}')
    loss_list3 = load_average_loss(f'awpinn/{config}')
    loss_list4 = load_average_loss(f'awipinn/{config}')
    
    fig = plt.figure(figsize=(12, 9))
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list1[:num_epochs]], color='blue', label='pinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list2[:num_epochs]], color='red', label='ipinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list3[:num_epochs]], color='green', label='awpinn')
    plt.plot(range(num_epochs), [np.log(item) for item in loss_list4[:num_epochs]], color='cyan', label='awipinn')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'test_plot/{config}_4layers.png', bbox_inches='tight')

# List of configurations to create plots for
configurations = ['1-1-1', '2-1-1']

# Create and save loss plots for each configuration
for config in configurations:
    create_loss_plot(config)

# test minimal

In [25]:
# Create a single figure with one row and three columns
pinns = pd.read_csv('Physics_Informed_Neural_Network/pinn_loss_hist.csv')
ipinns = pd.read_csv('Physics_Informed_Neural_Network/ipinn_loss_hist.csv')
awpinns = pd.read_csv('Physics_Informed_Neural_Network/awpinn_loss_hist.csv')
awipinns = pd.read_csv('Physics_Informed_Neural_Network/awipinn_loss_hist.csv')
plt.figure(figsize=(24, 32))


plt.subplot(421)
plt.plot(range(100000), [np.log(item) for item in pinns['training loss'][:100000]], color='blue', label='pinn')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)


plt.subplot(423)
plt.plot(range(100000), [np.log(item) for item in ipinns['training loss'][:100000]], color='red', label='ipinn')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)


plt.subplot(425)
plt.plot(range(100000), [np.log(item) for item in awpinns['training loss'][:100000]], color='green', label='awpinn')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)


plt.subplot(427)
plt.plot(range(100000), [np.log(item) for item in awipinns['training loss'][:100000]], color='yellow', label='awipinn')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)


plt.subplot(422)
plt.plot(range(100000), [np.log(item) for item in pinns['relative l2'][:100000]], color='blue', label='pinn')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)


plt.subplot(424)
plt.plot(range(100000), [np.log(item) for item in ipinns['relative l2'][:100000]], color='red', label='ipinn')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)


plt.subplot(426)
plt.plot(range(100000), [np.log(item) for item in awpinns['relative l2'][:100000]], color='green', label='awpinn')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)


plt.subplot(428)
plt.plot(range(100000), [np.log(item) for item in awipinns['relative l2'][:100000]], color='yellow', label='awipinn')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.grid(True)

# Adjust layout and display the figure
plt.tight_layout()
plt.show()